In [1]:
%matplotlib widget
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np
import cufflinks as cf
import glob


cf.go_offline()

sns.set(style="darkgrid")

In [2]:
data_paths = glob.glob('../PostHog/data/*.csv')
frames = [pd.read_csv(path, names=['title', 'language', 'icon', 'id', 'timestamp']) for path in data_paths]
data = pd.concat(frames)
data = data.drop_duplicates(subset=['id'])
data['timestamp'] = pd.to_datetime(data['timestamp'])
data['title'] = data['title'].str.strip()
print(len(data))
data.groupby(pd.Grouper(key='timestamp', axis=0, freq='1D'))['id'].count().iplot(kind='bar')

4543704


In [3]:
df = data.groupby(by=["icon", "title", "language"])[['id']].nunique()
print(len(df))
filtered = df[df['id'] >= 3]
print(len(filtered))
filtered = filtered.reset_index()
filtered.sort_values(by='id', ascending=False).head()

1475228
292009


,icon,title,language,id
2479,alarm.fill,Wake up!,en,196414
192811,moon.fill,Sleep Well!,en,88463
1051,alarm.fill,Aufwachen!,de,47513
88765,custom.shower,Shower,en,36230
209290,noun.meal,Lunch,en,21676


In [4]:
df.reset_index().groupby(by=["id"])[["id"]].count().iplot(kind='bar')

In [5]:
data.groupby(by=["icon"])["id"].count().sort_values(ascending=False).head(20)

icon
alarm.fill              376578
book.fill               241086
noun.meal               195234
mdi.school              190975
moon.fill               169724
custom.shower           139331
custom.dumbbell         137337
noun.coffee              90466
desktopcomputer          86379
heart.fill               84234
house.fill               78763
text.badge.checkmark     77597
wand.and.stars           75783
car.fill                 72719
briefcase.fill           64286
custom.cooking           60218
figure.walk              59897
custom.toothbrush        54713
person.3.fill            53319
sun.max.fill             51813
Name: id, dtype: int64

In [6]:
filtered.groupby(by=["language"])['id'].count().sort_values(ascending=False).iplot(kind='bar')

In [7]:
import json

for language in filtered["language"].drop_duplicates():
    export = filtered[filtered["language"] == language][['icon', 'title']]
    export.rename(columns = {'icon': 'label', 'title': 'text'}, inplace=True)
    export.to_csv(f'data/{language}.csv', index=False)
    print(language, len(export))
export = filtered[['icon', 'title']]
export.rename(columns = {'icon': 'label', 'title': 'text'}, inplace=True)
export.to_csv(f'data/all.csv', index=False)


en 163701
fr 12535
de 37396
es 9958
it 7807
pt 4515
nb 1466
ro 340
pl 6272
sv 2360
nl 11524
zh 12870
th 5190
ko 138
vi 1947
fi 552
ru 4056
ar 1019
ja 5184
tr 651
ca 241
id 468
hr 170
uk 385
da 1130
hu 102
fa 30
ur 2


In [8]:
export = filtered
export.rename(columns={'icon': 'label', 'title': 'text'}, inplace=True)
export = export[['label', 'text']]
export.to_json('user_icons_learn.json', orient="records")
